***GENERATED CODE FOR denial15oct2020 PIPELINE.***

***DON'T EDIT THIS CODE.***

***CONNECTOR FUNCTIONS TO READ DATA FROM DATABRICKS FILESYSTEM.***

In [ ]:
import warnings
warnings.filterwarnings('ignore')


class RDBMSConnector:

    def fetch(spark, config):
        drivers = {"mssql": "com.microsoft.sqlserver.jdbc.SQLServerDriver"}
        return spark.read.format("jdbc") \
            .option("url", f"jdbc:sqlserver://{eval(config)['host']}:{eval(config)['port']};databaseName={eval(config)['database']}") \
            .option(eval(config)['qtype'], eval(config)['query']) \
            .option("user", eval(config)['user']) \
            .option("password", eval(config)['password']) \
            .option("driver",  "com.microsoft.sqlserver.jdbc.SQLServerDriver") \
            .load()

    def put(df, spark, config):
        drivers = {"mssql": "com.microsoft.sqlserver.jdbc.SQLServerDriver"}
        # Write modes: overwrite, append
        df.write.mode(eval(config)['writemode'])\
            .format('jdbc') \
            .option("url", f"jdbc:{eval(config)['dbtype']}://{eval(config)['host']}:{eval(config)['port']};databaseName={eval(config)['database']}") \
            .option("dbtable", eval(config)['table']) \
            .option("user", eval(config)['user']) \
            .option("password", eval(config)['password']) \
            .option("driver",  "com.microsoft.sqlserver.jdbc.SQLServerDriver") \
            .save()


***OPERATION FUNCTIONS***

In [ ]:


def QueryEditor(spark, joindataobj):
    for source in joindataobj["sourcesForQuery"]:
        sourcename = source['sourcename']
        source['sourcename'] = spark.read.options(header='true', inferschema='true', delimiter=',')\
            .csv('dbfs:' + source['sourcepath'])
        source['sourcename'].createOrReplaceTempView(sourcename)
    joinedDf = spark.sql(joindataobj['querytext'])
    df = removeIfDuplicates(joinedDf)
    print(display(df.limit(2).toPandas()))
    return df


def removeIfDuplicates(joinedDf):
    cols_new = []
    seen = set()
    for c in joinedDf.columns:
        cols_new.append('{}_dup'.format(c) if c in seen else c)
        seen.add(c)
    df = joinedDf.toDF(*cols_new).select(*
                                         [c for c in cols_new if not c.endswith('_dup')])
    return df


***TRANSFORMATIONS FUNCTIONS THAT WILL BE APPLIED ON DATA***

In [ ]:
import json
from pyspark.sql.types import IntegerType
from pyspark.ml.feature import StringIndexer
from pyspark.sql.functions import col, when
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import mean, stddev, min, max, col


class CleanseData:
    # def __init__(self,df):
    #     #print()

    def replaceByMean(self, feature, df, mean_=-1):

        meanValue = df.select(mean(col(feature.name)).alias(
            'mean')).collect()[0]["mean"]
        df.fillna(meanValue, subset=[feature.name])
        df.withColumn(feature.name, when(col(feature.name) == " ",
                                         meanValue).otherwise(col(feature.name).cast("Integer")))
        return df

    def replaceByMax(self, feature, df, max_=-1):
        maxValue = df.select(max(col(feature.name)).alias('max')).collect()[
            0]["max"]
        df.fillna(maxValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", maxValue).otherwise(col(feature.name)))
        return df

    def replaceByMin(self, feature, df, min_=-1):
        minValue = df.select(min(col(feature.name)).alias('min')).collect()[
            0]["min"]
        df.fillna(minValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", minValue).otherwise(col(feature.name)))
        return df

    def replaceByStandardDeviation(self, feature, df, stddev_=-1):
        stddevValue = df.select(stddev(col(feature.name)).alias(
            'stddev')).collect()[0]["stddev"]
        df.fillna(stddevValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", stddevValue).otherwise(col(feature.name)))
        return df

    def replaceDateRandomly(self, feature, df):
        fillValue = df.where(col(feature.name).isNotNull()
                             ).head(1)[0][feature.name]
        df.fillna(str(fillValue), subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", fillValue).otherwise(col(feature.name)))
        # print("CleanseData:replaceDateRandomly Schema : ", df.#printSchema())
        return df

    def replaceNullValues(self, fList, df):
        featuresList = df.schema.fields
        for featureObj in fList:
            for feat in featuresList:
                if featureObj["feature"] in feat.name:
                    featureName = feat
                    if "mean" in featureObj["replaceby"]:
                        df = self.replaceByMean(featureName, df)
                    elif "max" in featureObj["replaceby"]:
                        df = self.replaceByMax(featureName, df)
                    elif "min" in featureObj["replaceby"]:
                        df = self.replaceByMin(featureName, df)
                    elif "stddev" in featureObj["replaceby"]:
                        df = self.replaceByStandardDeviation(featureName, df)
                    elif "random" in featureObj["replaceby"]:
                        df = self.replaceDateRandomly(featureName, df)
        return df


def StringIndexerTransform(df, params, transformationData={}):
    dfReturn = df
    feature = params["feature"]

    dfReturn = dfReturn.fillna({feature: ''})
    outcol = feature + "_stringindexer"
    indexer = StringIndexer(
        inputCol=feature, outputCol=outcol, handleInvalid="skip")
    indexed = indexer.fit(dfReturn).transform(dfReturn)
    dfReturn = indexed
    distinct_values_list = dfReturn.select(
        outcol).distinct().rdd.map(lambda r: r[0]).collect()
    len_distinct_values_list = len(distinct_values_list)
    if len_distinct_values_list <= 4:
        changed_type_df = dfReturn.withColumn(
            outcol, dfReturn[outcol].cast(IntegerType()))
        return changed_type_df
    return dfReturn


class TransformationMain:
    # TODO: change df argument in run with following
    def run(transformationDF, config):
        configObj = json.loads(config)
        featureData = configObj["FE"]
        transformationDF = CleanseData().replaceNullValues(featureData, transformationDF)
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'CPTCode', 'transformation_label': 'String Indexer'}], 'feature': 'CPTCode', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
                                                  'count': '10289', 'mean': '88698.17', 'stddev': '93788.0', 'min': '00731', 'max': 'V5299', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'CPTCode'}, {'feature_label': 'CPTCode', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('CPTCode')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'City', 'transformation_label': 'String Indexer'}], 'feature': 'City', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '10289', 'mean': '', 'stddev': '', 'min': 'Carrollton', 'max': 'Yorktown', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'City'}, {'feature_label': 'City', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('City')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'FacilityCategory', 'transformation_label': 'String Indexer'}], 'feature': 'FacilityCategory', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '10289', 'mean': '', 'stddev': '', 'min': 'After Hours Care Newport News', 'max': 'Yorktown Family Medicine', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'FacilityCategory'}, {'feature_label': 'FacilityCategory', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('FacilityCategory')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Gender', 'transformation_label': 'String Indexer'}], 'feature': 'Gender', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '10289', 'mean': '', 'stddev': '', 'min': 'F', 'max': 'M', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Gender'}, {'feature_label': 'Gender', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Gender')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'PracticeCategory', 'transformation_label': 'String Indexer'}], 'feature': 'PracticeCategory', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '10289', 'mean': '', 'stddev': '', 'min': 'Charles Lee Ginsburgh', 'max': 'TPMG', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'PracticeCategory'}, {'feature_label': 'PracticeCategory', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('PracticeCategory')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'DepartmentCategory', 'transformation_label': 'String Indexer'}], 'feature': 'DepartmentCategory', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '10289', 'mean': '', 'stddev': '', 'min': '<Unspecified>', 'max': 'X-Ray', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'DepartmentCategory'}, {'feature_label': 'DepartmentCategory', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('DepartmentCategory')
        display(transformationDF.limit(2).toPandas())
        return transformationDF


***AUTOML FUNCTIONS***

In [ ]:
from tpot import TPOTClassifier
from sklearn.model_selection import train_test_split
import pyspark


def functionClassification(sparkDF, listOfFeatures, label):
    sparkDF.persist(pyspark.StorageLevel.MEMORY_AND_DISK)
    df = (sparkDF.toPandas())
    X = (df.drop(label, axis=1))[listOfFeatures].values
    y = df[label].values
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, random_state=1, test_size=0.1)
    tpotModel = TPOTClassifier(verbosity=3, n_jobs=-1, generations=10, max_time_mins=5,
                               population_size=15)
    tpotModel.fit(X_train, y_train)
    display(" Accuracy of Model : %s" % tpotModel.score(X_test, y_test))
    data = {'model': tpotModel,
            'X_test': X_test,
            'y_test': y_test,
            'label': label,
            'columnNames': listOfFeatures}
    return data


***READING DATAFRAME***

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

#%run denial15oct2020Hooks.ipynb
try:
	#sourcePreExecutionHook()
	denial = RDBMSConnector.fetch(spark, "{'url': '/FileStore/platform/sampleData/5f8837d08b0e9ca211484fba/part-00000-tid-5206032533898413152-f0aec763-b60d-43fd-a32d-b74e3b6dbe9e-59684-1-c000.csv', 'host': 'wsc-sqlserver1.database.windows.net', 'port': '1433', 'password': 'G5gTW5rnKVeAZWXm', 'database': 'NextGen_Tidewater_Physicians', 'optionalDB': 'NextGen_Tidewater_Physicians', 'user': 'Numtra', 'qtype': 'query', 'query': "SELECT DISTINCT TOP(100000) cd.Unit, cd.Amount, cp.CPTCode, fac.City, fac.FacilityCategory, pat.Gender,\r\nFLOOR(DATEDIFF(DAY, pat.DateOfBirth, GETDATE()) / 365.25) AS Age, Pr.PracticeCategory, dep.DepartmentCategory,\r\ncd.PatientBal, cd.PrimaryInsuranceBal,  cd.DiagnosisCodeID1 AS Diagnosis1,\r\nMonth(cd.DateOfEntry) AS MonthOfEntry,  cd.DiagnosisCodeID2 AS Diagnosis2, \r\nstatus=(SELECT CASE trc.ReasonType WHEN 'Denial' THEN 1 ELSE 0 END)\r\nFROM chargedetail cd inner join TransactionReason as tr on tr.ChargeDetailID = cd.ChargeDetailID\r\ninner join TransactionReasonCode as trc on tr.TransactionReasonCodeID = trc.TransactionReasonCodeID \r\ninner join Practice as Pr on Pr.PracticeID = cd.PracticeID  \r\ninner join CptCode as cp on cp.CPTCodeID = cd.CPTCodeID  \r\ninner join Department as dep on dep.DepartmentID = cd.DepartmentID  \r\ninner join Facility as fac on fac.FacilityID = cd.FacilityID  \r\ninner join Patient as pat on pat.PatientID = cd.PatientID  \r\nWHERE trc.ReasonType IN ('Denial') \r\nand cd.Amount > .01 and\r\nexists ( select distinct ChargeDetailID as cd from TransactionReasonCode as tr inner join TransactionReason as r on tr.TransactionReasonCodeID = r.TransactionReasonCodeID and cd.ChargeDetailID = r.ChargeDetailID)", 'dbtype': 'mssql', 'is_header': 'Use Header Line'}")



	nondenial = RDBMSConnector.fetch(spark, "{'url': '/FileStore/platform/sampleData/5f8837d28b0e9ca211484fbb/part-00000-tid-6442739461415488559-d515570c-68cd-466c-9c2b-56699e927c9d-59688-1-c000.csv', 'host': 'wsc-sqlserver1.database.windows.net', 'port': '1433', 'password': 'G5gTW5rnKVeAZWXm', 'database': 'NextGen_Tidewater_Physicians', 'optionalDB': 'NextGen_Tidewater_Physicians', 'user': 'Numtra', 'qtype': 'query', 'query': "SELECT DISTINCT TOP(100000) cd.Unit, cd.Amount, cp.CPTCode, fac.City, fac.FacilityCategory, pat.Gender,\r\nFLOOR(DATEDIFF(DAY, pat.DateOfBirth, GETDATE()) / 365.25) AS Age, Pr.PracticeCategory, dep.DepartmentCategory,\r\ncd.PatientBal, cd.PrimaryInsuranceBal,  cd.DiagnosisCodeID1 AS Diagnosis1,\r\nMonth(cd.DateOfEntry) AS MonthOfEntry,  cd.DiagnosisCodeID2 AS Diagnosis2, \r\nstatus=(SELECT CASE trc.ReasonType WHEN 'Denial' THEN 1 ELSE 0 END)\r\nFROM chargedetail cd inner join TransactionReason as tr on tr.ChargeDetailID = cd.ChargeDetailID\r\ninner join TransactionReasonCode as trc on tr.TransactionReasonCodeID = trc.TransactionReasonCodeID \r\ninner join Practice as Pr on Pr.PracticeID = cd.PracticeID  \r\ninner join CptCode as cp on cp.CPTCodeID = cd.CPTCodeID  \r\ninner join Department as dep on dep.DepartmentID = cd.DepartmentID  \r\ninner join Facility as fac on fac.FacilityID = cd.FacilityID  \r\ninner join Patient as pat on pat.PatientID = cd.PatientID  \r\nWHERE trc.ReasonType IN ('Contractual', 'Informational', 'Co-Ins/Deduct', 'SPAY-Patient Responsibility', 'Other') \r\nand cd.Amount > .01 and\r\nexists ( select distinct ChargeDetailID as cd from TransactionReasonCode as tr inner join TransactionReason as r on tr.TransactionReasonCodeID = r.TransactionReasonCodeID and cd.ChargeDetailID = r.ChargeDetailID)", 'dbtype': 'mssql', 'is_header': 'Use Header Line'}")
	#sourcePostExecutionHook(nondenial)

except Exception as ex: 
	logging.error(ex)


***PERFORMING OPERATIONS***

In [ ]:
#%run denial15oct2020Hooks.ipynb
try:
	#operationPreExecutionHook()

	joindata = QueryEditor(spark, {"querytext": "SELECT  * FROM Denial UNION ALL Select *  FROM NonDenial ", "sourcesForQuery": [{"sourcename": "Denial", "sourcepath": "/FileStore/platform/sampleData/5f8837d08b0e9ca211484fba/part-00000-tid-5206032533898413152-f0aec763-b60d-43fd-a32d-b74e3b6dbe9e-59684-1-c000.csv", "sourceid": "5f8837d08b0e9ca211484fba"}, {"sourcename": "NonDenial", "sourcepath": "/FileStore/platform/sampleData/5f8837d28b0e9ca211484fbb/part-00000-tid-6442739461415488559-d515570c-68cd-466c-9c2b-56699e927c9d-59688-1-c000.csv", "sourceid": "5f8837d28b0e9ca211484fbb"}], "fieldsForQuery": []})
	#operationPostExecutionHook(joindata)

except Exception as ex: 
	logging.error(ex)


***TRANSFORMING DATAFRAME***

In [ ]:
#%run denial15oct2020Hooks.ipynb
try:
	#transformationPreExecutionHook()

	autofe = TransformationMain.run(joindata,json.dumps( {"FE": [{"transformationsData": [{"transformation_label": "novalue"}], "feature": "Unit", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "10289", "mean": "1.05", "stddev": "1.63", "min": "1.0", "max": "99.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Unit"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Amount", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "10289", "mean": "185.03", "stddev": "256.61", "min": "5.0", "max": "5838.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Amount"}, {"transformationsData": [{"feature_label": "CPTCode", "transformation_label": "String Indexer"}], "feature": "CPTCode", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "10289", "mean": "88698.17", "stddev": "93788.0", "min": "00731", "max": "V5299", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "CPTCode"}, {"transformationsData": [{"feature_label": "City", "transformation_label": "String Indexer"}], "feature": "City", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "10289", "mean": "", "stddev": "", "min": "Carrollton", "max": "Yorktown", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "City"}, {"transformationsData": [{"feature_label": "FacilityCategory", "transformation_label": "String Indexer"}], "feature": "FacilityCategory", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "10289", "mean": "", "stddev": "", "min": "After Hours Care Newport News", "max": "Yorktown Family Medicine", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "FacilityCategory"}, {"transformationsData": [{"feature_label": "Gender", "transformation_label": "String Indexer"}], "feature": "Gender", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "10289", "mean": "", "stddev": "", "min": "F", "max": "M", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Gender"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Age", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "10289", "mean": "62.87", "stddev": "18.32", "min": "0", "max": "120", "missing": "0"}, "updatedLabel": "Age"}, {"transformationsData": [{"feature_label": "PracticeCategory", "transformation_label": "String Indexer"}], "feature": "PracticeCategory", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "10289", "mean": "", "stddev": "", "min": "Charles Lee Ginsburgh", "max": "TPMG", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "PracticeCategory"}, {"transformationsData": [{"feature_label": "DepartmentCategory", "transformation_label": "String Indexer"}], "feature": "DepartmentCategory", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "10289", "mean": "", "stddev": "", "min": "<Unspecified>", "max": "X-Ray", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "DepartmentCategory"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "PatientBal", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "10289", "mean": "2.32", "stddev": "27.01", "min": "-336.17", "max": "1647.15", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "PatientBal"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "PrimaryInsuranceBal", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "10289", "mean": "1.92", "stddev": "38.76", "min": "-251.0", "max": "3150.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "PrimaryInsuranceBal"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Diagnosis1", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "10289", "mean": "33816.88", "stddev": "19073.11", "min": "391", "max": "94919", "missing": "0"}, "updatedLabel": "Diagnosis1"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "MonthOfEntry", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "10289", "mean": "5.96", "stddev": "3.5", "min": "1", "max": "12", "missing": "0"}, "updatedLabel": "MonthOfEntry"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Diagnosis2", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "10289", "mean": "36675.04", "stddev": "21652.04", "min": "849", "max": "94919", "missing": "0"}, "updatedLabel": "Diagnosis2"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "status", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "10289", "mean": "0.47", "stddev": "0.5", "min": "0", "max": "1", "missing": "0"}, "updatedLabel": "status"}, {"feature": "CPTCode_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "real", "selected": "True", "stats": {"count": "10289", "mean": "17.36", "stddev": "41.95", "min": "0.0", "max": "334.0", "missing": "0"}, "updatedLabel": "CPTCode_stringindexer_tra..."}, {"feature": "City_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "real", "selected": "True", "stats": {"count": "10289", "mean": "1.54", "stddev": "2.4", "min": "0.0", "max": "15.0", "missing": "0"}, "updatedLabel": "City_stringindexer_transf..."}, {"feature": "FacilityCategory_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "real", "selected": "True", "stats": {"count": "10289", "mean": "20.6", "stddev": "26.95", "min": "0.0", "max": "160.0", "missing": "0"}, "updatedLabel": "FacilityCategory_stringin..."}, {"feature": "Gender_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "numeric", "selected": "True", "stats": {"count": "10289", "mean": "0.42", "stddev": "0.49", "min": "0", "max": "1", "missing": "0"}, "updatedLabel": "Gender_stringindexer_tran..."}, {"feature": "PracticeCategory_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "numeric", "selected": "True", "stats": {"count": "10289", "mean": "0.01", "stddev": "0.1", "min": "0", "max": "1", "missing": "0"}, "updatedLabel": "PracticeCategory_stringin..."}, {"feature": "DepartmentCategory_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "real", "selected": "True", "stats": {"count": "10289", "mean": "1.06", "stddev": "1.68", "min": "0.0", "max": "24.0", "missing": "0"}, "updatedLabel": "DepartmentCategory_string..."}]}))

	#transformationPostExecutionHook(autofe)

except Exception as ex: 
	logging.error(ex)


***TRAIN MODEL***

In [ ]:
#%run denial15oct2020Hooks.ipynb
try:
	#mlPreExecutionHook()

	dataAutoML=functionClassification(autofe, ["Unit", "Amount", "CPTCode_stringindexer", "City_stringindexer", "FacilityCategory_stringindexer", "Gender_stringindexer", "Age", "PracticeCategory_stringindexer", "DepartmentCategory_stringindexer", "PatientBal", "PrimaryInsuranceBal", "Diagnosis1", "MonthOfEntry", "Diagnosis2"], "status")

	#mlPostExecutionHook(dataAutoML)

except Exception as ex: 
	logging.error(ex)


***PREDICT ON TRAINED MODEL***

In [ ]:
import pandas as pd
import numpy as np
import sklearn.metrics

try:
    model=dataAutoML['model']
    X_test=dataAutoML['X_test']
    y_test=dataAutoML['y_test']
    label=dataAutoML['label']
    columnNames=dataAutoML['columnNames']
    if label in columnNames:
        columnNames.remove(label)
    predicted=label+"_predicted"
    y_predicted=model.predict(X_test)
    df =pd.DataFrame(X_test , columns=columnNames)
    df[label]=y_test
    df[predicted]=y_predicted
    columnNames.insert(0,predicted)
    columnNames.insert(0,label)
    Accuracy = np.round((100 * sklearn.metrics.accuracy_score(y_true=y_test, y_pred=y_predicted)), 1)
    F1= np.round(
            (100 * sklearn.metrics.f1_score(y_true=y_test, y_pred=y_predicted, average="weighted")), 1)
    Precision= np.round((
                100 * sklearn.metrics.precision_score(y_true=y_test, y_pred=y_predicted, average="weighted")), 1)
    Recall = np.round((
                100 * sklearn.metrics.recall_score(y_true=y_test, y_pred=y_predicted, average="weighted")), 1)
    display(" Accuracy of Prediction on test data    : %s"%Accuracy)
    display(" F1 score of Prediction on test data    : %s"%F1)
    display(" Precision of Prediction on test data   : %s"%Precision)
    display(" Recall of Prediction on test data      : %s"%Recall)
    display(df.head())
except Exception as ex:
    logging.error(ex)

